This is the code that will be imported into our Raspberry Pi to process and send images to our computer for classification

In [4]:
# Importing packages
import numpy as np
import cv2
from skimage import data, filters

# Open Video
cap = cv2.VideoCapture(0)

while(True):
        # Capture video frame by frame
        ret, frame = cap.read()
        print("read is ", ret)
        print(frame)
        # Display frame
        cv2.imshow('frame', frame)
        
        #Press escape to close window
        c = cv2.waitKey(1)
        if c == 27:
            break

read is  False
None


[ WARN:0@137.901] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@137.901] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
